In [7]:
import psycopg2
from psycopg2.extras import execute_values
import faker
from faker import Faker
import random
from datetime import timedelta

fake = Faker("vi_VN")

In [8]:
conn = psycopg2.connect(
    host="aws-1-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="postgres.rruavcjmtgpxyznzwkhw",
    password="khaibaolocnguyen",
    port=5432
)
cur = conn.cursor()

cur.execute("SELECT branch_id FROM branch")

branches = cur.fetchall()
branches = [b[0] for b in branches]
print(branches)

['015cf519-76c9-478e-9bf8-e25bb91f92bb', '8adb1159-cee3-4c59-b08f-3a9aa369d73e', '29e07c3a-47bf-4fef-8fbe-471e1587118c', 'a49d5373-fdae-4e7e-b90f-17344d0a6aaf', 'c26300f2-703c-4022-82c0-b68dc260aa4f', 'ad3cc27c-6b13-46ed-bcfd-cc099f0d5d47', '6d1f53af-555c-4296-9066-e304432d0633', '6feeeb11-f446-46da-9214-cd004151c492', '7192a064-1e01-42c7-87b9-6e6c3d381339', '0e00def4-2be9-405f-bb3e-1a2bc888828b']


In [9]:
conn.rollback()

cur.execute("""
  select employee_id
  from account
  join employee on account.account_id = employee.account_id
  where password = 'User@123'
""")

employee_ids = cur.fetchall()
employee_ids = [e[0] for e in employee_ids]
print(employee_ids)

['ba23f7df-071f-4de7-ae75-16e9e54cff18', 'd49259a6-9bca-4495-9dfe-4361f43c9328', '68f1afd2-1471-4d5b-9b66-e16e923c9f5d', '8324bb52-aaa0-42db-8833-024b9afed652', 'c9a54ed4-b78b-4a21-bfed-9d81209d5e99', '10599160-3c76-4e77-b006-6a63f18aaa02', '43b25694-c174-47b7-bf38-9f61c8705a89', '06250efc-fd46-43ec-b789-f4a13538b4c6', '397c6165-e76d-49ec-ab75-86d6f0a66303', 'da852ad7-610f-4e07-a84c-d7a7a8efda98', 'acb351cb-dfae-495b-948d-1a847f9bf666', '5f6e69f6-889c-4b44-8f30-eeb92664c3f9', 'f106a448-f965-4d12-9997-1d08f499c27c', '97279adc-457d-462f-b6a6-e57a201b787e', '54b17e40-e149-4b5f-9a0c-d86aca35ef8d', '709193a6-a70c-4134-9de0-c53d9c94aa16', 'b5bbac6a-b636-4898-a79f-547be56a5773', '49da80ed-55ff-4246-8c08-36d09c595677', 'cf7b7682-b90f-49a3-a2f2-2f91b2ecde83', '635d6c07-d42c-4333-aa6a-5ed35ab3324c', '142d238f-1fe9-4e8d-b97e-929921d5876e', 'c4cf9353-e4cb-41a4-921e-82535a08d12d', 'fcbbfff2-e579-41e9-a5bc-4cca4bd174c2', 'cb3aad2c-b8b7-4d36-8715-e10c4a3077b5', '21d8f6f1-6142-46bf-a54a-778e6d436f58',

In [10]:
history = []

for employee_id in employee_ids:
    num_history = random.randint(2, 3)

    # ngày bắt đầu đầu tiên (quá khứ)
    start_date = fake.date_between(start_date='-3y', end_date='-1y')

    for i in range(num_history):
        branch_id = random.choice(branches)
        position = random.choice(['staff', 'manager', 'supervisor'])
        salary = random.randint(5_000_000, 20_000_000)

        # history cuối cùng
        if i == num_history - 1:
            end_date = None
        else:
            # mỗi lần cách nhau 6–18 tháng
            duration = random.randint(180, 540)
            end_date = start_date + timedelta(days=duration)

        history.append({
            'branch_id': branch_id,
            'employee_id': employee_id,
            'start_date': start_date,
            'end_date': end_date,
            'position': position,
            'salary': salary
        })

        # chuẩn bị cho history tiếp theo
        if end_date:
            start_date = end_date + timedelta(days=1)

print(len(history))

732


In [11]:
query = """
INSERT INTO employeehistory (employee_id, branch_id, start_date, position, salary)
VALUES %s
"""

values = [
    (a["employee_id"], a["branch_id"], a["start_date"], a["position"], a["salary"])
    for a in history
]

execute_values(cur, query, values, page_size=len(values))

conn.commit()